# HPGe 刻度

In [1]:
%jsroot on

In [2]:
TCanvas *c1 = new TCanvas("c1");
TFile *filer = new TFile("gamma.root","READ");// 以只读模式打开ROOT文件
if(!filer->IsOpen())
  {
    std::cout<<"Can't open root file"<<std::endl;
  }

TH1F *h_gammaraw = (TH1F*)filer->Get("h0");//通过指针读取原始gamma谱
// 通过指针 h_gammaraw 可对直方图进行操作

h_gammaraw->Draw();
c1->Draw();

# 对能谱进行粗略的能量刻度
由于<sup>152</sup>Eu的344.28 KeV和<sup>133</sup>Ba的356 keV两条$\gamma$强度很大且离的较近，在能谱中很容易识别，选取这两条$\gamma$进行粗略刻度，找到其他$\gamma$再进行详细刻度。

In [3]:
h_gammaraw->Fit("gaus","","",332,334);// 这里 1215 为拟合的左边界，1219 为拟合的右边界
h_gammaraw->Draw();
h_gammaraw->GetXaxis()->SetRangeUser(325,340);//设置显示的区间
c1->Draw();

 FCN=2086.63 FROM MIGRAD    STATUS=CONVERGED      77 CALLS          78 TOTAL
                     EDM=7.79848e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   3.6 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.01943e+06   5.02073e+02  -9.26014e+00  -9.70224e-07
   2  Mean         3.32764e+02   4.25814e-04   4.89280e-06  -1.01689e+00
   3  Sigma        6.90517e-01   4.76922e-04   2.74589e-06  -1.07768e+00


In [4]:
h_gammaraw->GetXaxis()->SetRangeUser(0,2500);//返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题
h_gammaraw->Fit("gaus","","",322,324);//指定拟合的左右边界， 在Fit函数的第二个参数设为"L"时进行最大似然法拟合
h_gammaraw->Draw();
h_gammaraw->GetXaxis()->SetRangeUser(315,330);//设置显示的区间
c1->Draw();

 FCN=3780.64 FROM MIGRAD    STATUS=CONVERGED      66 CALLS          67 TOTAL
                     EDM=3.04318e-08    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   3.5 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     5.60434e+05   3.80324e+02   4.77816e+00   4.65939e-08
   2  Mean         3.22893e+02   4.97171e-04  -2.56381e-06  -4.56004e-01
   3  Sigma        7.01171e-01   6.52003e-04  -2.84321e-06   5.23601e-01


In [5]:
TGraphErrors *graph = new TGraphErrors();
graph->SetPoint(0, 323, 344);
graph->SetPointError(0, 0.0005, 0); // 为第一个点设置误差 (0.0005, 0),误差太小，看不见
graph->SetPoint(1, 333, 356);
graph->SetPointError(1, 0.0004, 0); // 为第二个点设置误差 (0.0004, 0)
graph->SetMarkerStyle(20);
graph->Draw("AP");
graph->Fit("pol1");
c1->Draw();

 FCN=8.21759e-08 FROM MIGRAD    STATUS=CONVERGED      80 CALLS          81 TOTAL
                     EDM=1.58731e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0          -4.36000e+01   2.51747e-02   2.07901e-05  -5.10357e-01
   2  p1           1.20000e+00   7.64878e-05   5.72205e-07  -1.61028e+02


In [6]:
    double p0 = -43.6;//填入拟合得到的数值
    double p1 = 1.2;
    TH1D *h1 = new TH1D("h1","",10000,0,2500);//h1为刻度后的gamma谱
    TRandom3 *r = new TRandom3(0);
    int Nbins = h0->GetXaxis()->GetNbins(); //h0为未刻度的gamma谱。
    for(int i=0; i<Nbins; i++)
    {
        
        Long64_t eN = h0->GetBinContent(i);// GetBinContent()提取每个bin的计数
        Double_t e = h0->GetBinLowEdge(i);// GetBinLowEdge()提取每个bin左边界的横坐标x。
        for(Long64_t j=0; j<eN; j++)
        {
            Double_t ea = e+r->Rndm()*0.2;// 思考下为何要加上一个随机数？？？？？？？？？？？？？？
            ea=p1*ea+p0;//p0,p1,p2为上面获得的二次项刻度系数。
            h1->Fill(ea);
        }
    }
h1->Draw();
c1->Draw();

## 选取更多能量点进一步拟合

In [7]:
h1->GetXaxis()->SetRangeUser(0,2500);//返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题
h1->Fit("gaus","","",301,303);//302.9 keV
h1->Draw();
h1->GetXaxis()->SetRangeUser(290,310);//设置显示的区间
c1->Draw();

 FCN=996.385 FROM MIGRAD    STATUS=CONVERGED      89 CALLS          90 TOTAL
                     EDM=6.94157e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     3.63630e+05   3.37164e+02   3.65603e+00   1.10711e-06
   2  Mean         3.02054e+02   8.94063e-04   1.44030e-04   1.06855e-01
   3  Sigma        8.39672e-01   1.43521e-03   8.01858e-06   6.52491e-01


In [8]:
h1->GetXaxis()->SetRangeUser(0,2500);//返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题
h1->Fit("gaus","","",274,277);//276.4 keV
h1->Draw();
h1->GetXaxis()->SetRangeUser(260,280);//设置显示的区间
c1->Draw();

 FCN=1273.88 FROM MIGRAD    STATUS=CONVERGED      65 CALLS          66 TOTAL
                     EDM=4.13202e-09    STRATEGY= 1  ERROR MATRIX UNCERTAINTY   1.9 per cent
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.66576e+05   1.93384e+02   6.71628e-01  -3.51407e-07
   2  Mean         2.75393e+02   1.12142e-03   7.48932e-07  -1.42391e-02
   3  Sigma        9.89008e-01   1.43797e-03  -1.82922e-06   1.10936e-01


In [9]:
h1->GetXaxis()->SetRangeUser(0,2500);//返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题
h1->Fit("gaus","","",118,121);//121.78 keV
h1->Draw();
h1->GetXaxis()->SetRangeUser(110,130);//设置显示的区间
c1->Draw();

 FCN=5582.79 FROM MIGRAD    STATUS=CONVERGED      74 CALLS          75 TOTAL
                     EDM=1.22426e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     8.42950e+05   4.45574e+02   1.19425e+01   5.40206e-07
   2  Mean         1.19554e+02   3.99195e-04   5.70076e-05  -9.38449e-01
   3  Sigma        8.46508e-01   4.44821e-04   5.23207e-06   2.33540e+00


In [10]:
h1->GetXaxis()->SetRangeUser(0,2500);//返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题
h1->Fit("gaus","","",780,784);//778.9 keV
h1->Draw();
h1->GetXaxis()->SetRangeUser(770,790);//设置显示的区间
c1->Draw();

 FCN=2104.74 FROM MIGRAD    STATUS=CONVERGED      72 CALLS          73 TOTAL
                     EDM=1.91651e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.67596e+05   1.76156e+02   2.95681e+00  -3.31021e-06
   2  Mean         7.82235e+02   9.33688e-04   3.72999e-04  -3.11428e-01
   3  Sigma        1.01429e+00   9.41885e-04   5.55100e-06  -1.91615e+00


In [11]:
h1->GetXaxis()->SetRangeUser(0,2500);//返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题
h1->Fit("gaus","","",967,971);//964.08 keV
h1->Draw();
h1->GetXaxis()->SetRangeUser(950,990);//设置显示的区间
c1->Draw();

 FCN=813.628 FROM MIGRAD    STATUS=CONVERGED      62 CALLS          63 TOTAL
                     EDM=3.99637e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.62852e+05   1.71196e+02   1.78753e+00   1.02829e-06
   2  Mean         9.69018e+02   9.54896e-04   4.62064e-04   2.54315e-01
   3  Sigma        1.04415e+00   9.79209e-04   3.54854e-06   1.83140e-01


In [12]:
h1->GetXaxis()->SetRangeUser(0,2500);//返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题
h1->Fit("gaus","","",1116,1120);//1212.95 keV
h1->Draw();
h1->GetXaxis()->SetRangeUser(1100,1130);//设置显示的区间
c1->Draw();

 FCN=1725.98 FROM MIGRAD    STATUS=CONVERGED      71 CALLS          72 TOTAL
                     EDM=6.44131e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.35329e+05   1.55042e+02   2.34148e+00   2.26478e-06
   2  Mean         1.11828e+03   1.15713e-03   5.33236e-04  -6.59449e-01
   3  Sigma        1.09732e+00   1.22558e-03   6.08111e-06   3.03565e+00


In [13]:
h1->GetXaxis()->SetRangeUser(0,2500);//返回全区间，在多次进行拟合时，必须要有这个操作，否则指定的拟合区间将存在问题
h1->Fit("gaus","","",1413,1420);//1408 keV
h1->Draw();
h1->GetXaxis()->SetRangeUser(1400,1430);//设置显示的区间
c1->Draw();

 FCN=11718.6 FROM MIGRAD    STATUS=CONVERGED      68 CALLS          69 TOTAL
                     EDM=1.98382e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.65809e+05   1.52996e+02   6.25325e+00   4.48242e-08
   2  Mean         1.41683e+03   8.68082e-04   6.75596e-04   1.42797e-02
   3  Sigma        1.18630e+00   7.19484e-04   8.25626e-06   1.00811e-01


In [14]:
TGraph *graph1 = new TGraph();
graph1->SetPoint(0, 119.6, 121.78);
graph1->SetPoint(1, 275.4, 276.4);
graph1->SetPoint(2, 302, 302.9);
graph1->SetPoint(3, 782.2, 778.9);
graph1->SetPoint(4, 969, 964.08);
graph1->SetPoint(5, 1118.3, 1213);
graph1->SetPoint(6, 1416.8, 1408);
graph1->SetMarkerStyle(20);
graph1->Draw("AP");
TF1 *fitFunc = new TF1("fitFunc", "pol1", 0, 1600);
graph1->Fit("fitFunc");
c1->Draw();


****************************************
Minimizer is Linear
Chi2                      =      7561.62
NDf                       =            5
p0                        =     -2.47917   +/-   27.3385     
p1                        =      1.01989   +/-   0.0323796   


In [15]:
TF1 *fitFunc2 = new TF1("fitFunc2", "pol2", 0, 1600);
graph1->Fit("fitFunc2");
c1->Draw();


****************************************
Minimizer is Linear
Chi2                      =      7320.82
NDf                       =            4
p0                        =     -15.3782   +/-   46.5735     
p1                        =      1.07626   +/-   0.159431    
p2                        = -3.81971e-05   +/-   0.000105305 


In [16]:
    double p10 = -2.47917;//填入pol1详细拟合得到的数值
    double p11 = 1.01989;
    TH1D *h11 = new TH1D("h11","",10000,0,2500);//h11为刻度后的gamma谱
    int Nbins1 = h1->GetXaxis()->GetNbins(); //h1为粗略刻度的gamma谱。
    for(int i=0; i<Nbins1; i++)
    {
        
        Long64_t eN = h1->GetBinContent(i);// GetBinContent()提取每个bin的计数
        Double_t e = h1->GetBinLowEdge(i);// GetBinLowEdge()提取每个bin左边界的横坐标x。
        for(Long64_t j=0; j<eN; j++)
        {
            Double_t ea = e+r->Rndm()*0.2;// 思考下为何要加上一个随机数？？？？？？？？？？？？？？
            ea=p11*ea+p10;//p0,p1,p2为上面获得的二次项刻度系数。
            h11->Fill(ea);
        }
    }
h11->Draw();
c1->Draw();

In [17]:
    double p20 = -15.3782;//填入pol2详细拟合得到的数值
    double p21 = 1.07626;
    double p22 = -3.81971e-05;
    TH1D *h21 = new TH1D("h21","",10000,0,2500);//h21为刻度后的gamma谱
    for(int i=0; i<Nbins1; i++)
    {
        
        Long64_t eN = h1->GetBinContent(i);// GetBinContent()提取每个bin的计数
        Double_t e = h1->GetBinLowEdge(i);// GetBinLowEdge()提取每个bin左边界的横坐标x。
        for(Long64_t j=0; j<eN; j++)
        {
            Double_t ea = e+r->Rndm()*0.2;// 思考下为何要加上一个随机数？？？？？？？？？？？？？？
            ea=p22*ea*ea+p21*ea+p20;//p0,p1,p2为上面获得的二次项刻度系数。
            h21->Fill(ea);
        }
    }
h21->Draw();
c1->Draw();